# SIR Modeling for COVID-19

Objectives: Look at the rate of COVID-19 growth by different regions and estimate the SIR curve.

* The recovery rate and overall fatality rate should be fixed.
* The infection rate should depend on local population density and social distancing measures (unique to each region).

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy import optimize
import statsmodels.api as sm
import os
import pickle
import requests

# Load Covid-19 and Census Data

In [ ]:
with open('./data/us_combined_df.pkl', 'rb') as f:
    us_combined_df = pickle.load(f)

# Load covid and world population data
with open('./data/world_combined_df.pkl', 'rb') as f:
    world_combined_df = pickle.load(f)
    
county_census_df = pd.read_csv('./data/co-est2019-alldata.csv', encoding='latin-1')

# Utility functions

Functions that are called to generate an SIR model, plot the curve, compute objective functions, etc.

In [ ]:
# Several utility functions for SIR model

def compute_sir(sampling_rate, total_days, pop, infected, contacts_per_day, days_to_recover):
    """Simulates the SIR output over a number of days using small fraction-of-day time steps.
    
    Args:
        sampling_rate: The number of samples per day.
        total_days: The total time to simulate.
        pop: The population of the area we are simulating.
        infected: The starting number of infected individuals.
        contacts_per_day: The number of people each infected individual infects per day
          at the start of simulation (nearly everyone is susceptible).
        days_to_recover: The number of days it takes someone to recover from the disease.
        
    Returns:
        Time indices, and associated susceptible, infected, and recovered populations
        
    """
    s = [1.0]
    i = [float(infected) / pop]
    r = [0.0]
    beta = contacts_per_day
    gamma = 1.0 / days_to_recover
    dt = 1.0 / sampling_rate
    for t in np.arange(0, total_days-dt, dt):
        prev_s = s[-1]
        prev_i = i[-1]
        prev_r = r[-1]
        # First order modeling
        s.append(prev_s - beta * prev_s * prev_i * dt)
        i.append(prev_i + (beta * prev_s - gamma) * prev_i * dt)
        r.append(prev_r + gamma * prev_i * dt)
        
    s = np.array(s)
    i = np.array(i)
    r = np.array(r)
    return np.arange(0, total_days, dt), pop * s, pop * i, pop * r


def create_mse_objective_fn(deaths, population, sampling_rate, starting_cases):
    """Create an objective function for Bayesian optimization using MSE of model vs actual data.
    
    Args:
      data: A numpy array with the raw daily data to model.
      population: The total population of the area to model.
      sampling_rate: The sampling_rate for the model in number of samples per day.
    
    Returns:
      A function that takes tuples for ranges of death_rate, contacts_per_day, and days_to_recover
      and returns -mse(model, actual) as a function to maximize.
    """
    def _fn(death_rate, contacts_per_day, days_to_recover, case_fatality_rate_multiplier):
        infected = 0.04 * starting_cases * case_fatality_rate_multiplier # Starting population infected = case fatality rate
        t, s, i, r = compute_sir(
            sampling_rate,
            len(deaths),
            population * death_rate,
            infected,
            contacts_per_day,
            days_to_recover
        )
        mse = np.mean(np.square(data - r[::sampling_rate]))
        return mse
    return _fn


def plot_sir_model(r, i, total_days, df, sampling_rate, name):
    """Plot the model death rates and total deaths vs actual data.
    
    Args:
        r: Array holding recovery values
        df: Dataframe holding death values.
        sampling_rate: Number of samples per day used to simulate the model.
    """
    plot_start_time = df['Date'].min().timestamp()
    plot_step_size = 24 * 60 * 60 / sampling_rate
    plot_end_time = plot_start_time + total_days * 24 * 60 * 60 
    plot_timestamps = np.arange(plot_start_time, plot_end_time, plot_step_size)
    plot_dates = [datetime.utcfromtimestamp(x) for x in plot_timestamps]
    print('peak date', plot_dates[np.argmax(i)])
    # Plot peak infection
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.ticklabel_format(useOffset=False)
    ax.ticklabel_format(style='plain')
    ax.plot(plot_dates[:-sampling_rate],
            (r[sampling_rate:] - r[:-sampling_rate]),
            c='g',
            label='model death rate',
            linewidth=4)
    ax.plot(df['Date'].to_list()[:-1],
            (df['Deaths'] - df['Deaths'].shift())[1:], label='actual death rate', c='r', linewidth=4)
    ax.set_title('SIR model for ' + name)
    ax.set_xlabel('Number of days')
    ax.set_ylabel('Number of individuals')
    plt.legend()
    plt.plot()
    
    # Plot recovery
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.ticklabel_format(useOffset=False)
    ax.ticklabel_format(style='plain')
    ax.plot(plot_dates, r, c='g',
            label='model deaths', linewidth=4)
    ax.plot(df['Date'].to_list(), df['Deaths'], label='actual deaths', c='r', linewidth=4)
    ax.set_title('SIR model for ' + name)
    ax.set_xlabel('Number of days')
    ax.set_ylabel('Number of individuals')
    plt.legend()
    plt.show()


def get_time_series_for_area(area_of_interest_spec):
    """Slice the dataframe by the are of interest and aggregate confirmed cases and deaths by date.
    
    NOTE: this function only works if you have already prefetched all of the data in the above cell!
    
    Args:
      area_of_interest_spec: A tuple (country, area_name, state_fips, county_fips).
    
    Returns:
      A Dataframe holding a time series of confirmed cases and deaths for the area of interest.
    """
    country = area_of_interest_spec[0]
    title = area_of_interest_spec[1]
    if country == 'US':
        state_fips = area_of_interest_spec[2]
        county_fips = area_of_interest_spec[3]

        if not county_fips:
            population = county_census_df[(county_census_df.STATE == state_fips)
                                          & (county_census_df.COUNTY == 0)]['POPESTIMATE2019'].sum()
            area_df = us_combined_df[
                (us_combined_df.FIPS > state_fips * 1000)
                & (us_combined_df.FIPS < (state_fips + 1) * 1000)].groupby('Date').agg(
                {'Cases': 'sum',
                 'Deaths': 'sum',
                })
        else:
            combined_fips = [state_fips * 1000 + y for y in county_fips]
            population = county_census_df[(county_census_df.STATE == state_fips)
                                          & (county_census_df.COUNTY.isin(county_fips))]['POPESTIMATE2019'].sum()
            area_df = us_combined_df[(us_combined_df.FIPS.isin(combined_fips))].groupby('Date').agg({
                'Cases': 'sum',
                'Deaths': 'sum',
            })
        area_df = area_df.reset_index()
        area_df = area_df.drop_duplicates(['Date'], keep='last')
    else:
        area_df = world_combined_df[(world_combined_df.Country_Region == country)]
        area_df = area_df.drop_duplicates(['Date', 'Country_Region'], keep='last')
        population = area_df['Population'].iloc[0]
    print('Total population', population)
    return area_df, population

# SIR model for large counties

A simple [SIR model](https://mathworld.wolfram.com/Kermack-McKendrickModel.html) can be used to model the spread of infections across a population. The SIR model maintains 3 different population subgroups: 

* **(S)usceptible**: Number of individuals who are susceptible to getting the infection.
* **(I)nfected**: Number of individuals currently with the disease (or currently contagious).
* **(R)ecovered**: Number of individuals who have recovered from the disease and are immune and no longer contagious.<sup>+</sup>

The differential equations guiding the SIR process over time (t) is:

$$
\begin{matrix}
\frac{dS}{dt} = -\beta SI \\
\frac{dI}{dt} = \beta SI - \gamma I \\
\frac{dR}{dt} = \gamma I \\
\end{matrix}
$$

One way to conceptualize the SIR model is a set of 3 buckets representing the populations. $\frac{dS}{dt}$ is the rate of growth in S, which should be negative since the population will continue to get infected. $-\beta SI$ represents the rate at which people get infected. If you imagine S susceptible individuals each mingling randomly with I infected individuals, this is why $SI$ exists. Beta represents the rate of disease transmission between individuals, i.e. the chance that the disease is transmitted between a random susceptible person to a random infected person. $\beta SI$ term exists in the second line because this is the negative of $\frac{dS}{dt}$, i.e. the rate of susceptible population getting infected. Additionally, $-\gamma I$, where $\gamma$ is the recovery rate, indicates that infected individuals will gradually lessen as they recover (or die) and become part of the recovered group. $\frac{dR}{dt}$ is simply the recovery rate.


## Model Simulation Details

We use first order Taylor expansion to simulate the process using approximations at every fractional time step. Note that higher order terms can be derived by applying the chain rule and substituting in the differential equations above, but with time steps much smaller than contact and recovery times, a first order approximator is probably sufficient for an accurate model.

$$
\begin{matrix}
S(t+\delta) &= S(t) &- \beta S(t)I(t)\delta &+ ...\\
I(t+\delta) &= I(t) &+ (\beta S(t) - \gamma)I(t)\delta &+ ...\\
R(t+\delta) &= R(t) &+ \gamma I(t)\delta &+... \\
\end{matrix}
$$

<sup>+</sup> Because death rates are very low compared to the overall population, we do not need to consider a 4th metric (i.e. death) here, but essentially deaths could be modeled as a fraction of the "recovered" bucket as they are no longer contagious.

In [ ]:
# Test out a model plot
sampling_rate = 10.0 # Times per day
total_days = 180 # Total number of days
population = 327.2 # Population in the US in millions
infected = 0.04 # Starting population dead = cases * case fatality rate
contacts_per_day = 0.2 # Average contacts per day for people
days_to_recover = 24 # Average days to recover (Avg 10 days until symptoms + 14 days after symptoms fade)

t, s, i, r = compute_sir(sampling_rate, total_days, population, infected, contacts_per_day, days_to_recover)

fig, ax = plt.subplots(figsize=(12, 8))
ax.ticklabel_format(useOffset=False)
ax.ticklabel_format(style='plain')
ax.plot(t, s, c='y', label='susceptible', linewidth=4)
ax.plot(t, i, c='r', label='infected', linewidth=4)
ax.plot(t, r, c='g', label='recovered', linewidth=4)
ax.set_title('SIR model')
ax.set_xlabel('Number of days')
ax.set_ylabel('Number of individuals (in millions)')
plt.legend()
plt.show()


In [ ]:
# Example on looing up state and county FIPS

lookup_df = county_census_df[(county_census_df.STNAME == 'Illinois') &
                             (county_census_df.CTYNAME.isin([
                                 'Cook County',
                                 'DeKalb County',
                                 'DuPage County',
                                 'Grundy County',
                                 'Kankakee County',
                                 'Kane County',
                                 'Kendall County',
                                 'McHenry County',
                                 'Will County',
                             ]))]
print(lookup_df['STATE'].iloc[0], ',', lookup_df['COUNTY'].tolist())

## Modeling parameters



In [ ]:
SIMULATION_DAYS = 120 # Total number of days to simulate when plotting forecast model.
SAMPLING_RATE = 10 # Modeling time samples per day

## Regions

Some interesting areas (Name, State FIPS, County FIPS) below. Copy one of the values in the bullet points into AREA_OF_INTEREST below.
* ('US', 'NYC', 36, [5, 47, 61, 81, 85])
* ('US', 'New Orleans', 22, [51, 71, 75, 87, 89, 95, 103, 105])
* ('US', 'Detroit', 26, [87, 93, 99, 125, 147, 163])
* ('US', 'Bay Area, CA', 6, [1, 13, 41, 55, 75, 81, 85, 95, 97])
* ('US', 'Greater LA Area, CA', 6, [37, 59, 65, 71, 111])
* ('US', 'Chicago', 17, [31, 37, 43, 63, 89, 91, 93, 111, 197])

If County FIPS is empty, this will fetch stats for the whole state:
* ('US', 'California', 6, [])
* ('US', 'New York', 36, [])
* ('US', 'Washington', 53, [])

If Country is not US, this will fetch a country's total stats:
* ('Italy', 'Italy')
* ('Spain', 'Spain')
* ('Germany', 'Germany') : A rapid recovery in Germany will fool the model because it's expecting the same rate of infection throughout.
* ('France', 'France')


In [ ]:
AREA_OF_INTEREST = ('Italy', 'Italy')
#AREA_OF_INTEREST = ('US', 'Washington', 53, [])
#AREA_OF_INTEREST = ('US', 'NYC', 36, [5, 47, 61, 81, 85])
MODEL_FIT_LAST_DATE = '2020-04-08'  # Fit model to data before this date, reserving later dates as holdout.

area_df, population = get_time_series_for_area(AREA_OF_INTEREST)
area_df = area_df[area_df.Date <= MODEL_FIT_LAST_DATE]
# Validate selection through plot and inspection
plt.plot(area_df['Date'], area_df['Deaths'])
area_df.tail() # Check last entries (Make sure data is good first!)

## Fitting Values to the Model

We try to find the best fit of all parameters of the model by minimizing its mean squared error (mse) from actual data points.

Note that the simple algorithm used below is randomized and not guaranteed to be optimal, but in practice, seems to converge to a near optimal solution quickly. Also, approaches such as Bayesian optimization, annealing, and other guaranteed optimal techniques take a long time to run per iteration and have occasionally stalled the notebook.

In [ ]:
# Initial random exploration

INIT_RANDOM_PTS = 10000 # Randomly sample this many points before starting optimization

# Reasonable search regions for each parameter
initial_death_rate_interval = (0.0001, 0.02)
initial_contacts_per_day_interval = (0.05, 2.0)
initial_days_to_recover_interval = (5.0, 50.0)
initial_case_fatality_multiplier_interval = (0.05, 100.0)

data = area_df['Deaths'].to_numpy()
starting_cases = area_df['Cases'].min()

mse_metric = create_mse_objective_fn(data, population, SAMPLING_RATE, starting_cases)


min_bounds = np.array([initial_death_rate_interval[0],
                       initial_contacts_per_day_interval[0],
                       initial_days_to_recover_interval[0],
                       initial_case_fatality_multiplier_interval[0],
                      ])
max_bounds = np.array([initial_death_rate_interval[1],
                       initial_contacts_per_day_interval[1],
                       initial_days_to_recover_interval[1],
                       initial_case_fatality_multiplier_interval[1],
                      ])

death_rate_interval = initial_death_rate_interval

# Initially randomly sample a large number of points
params = np.random.uniform(min_bounds, max_bounds, size=(INIT_RANDOM_PTS, 4))

values = [mse_metric(row[0], row[1], row[2], row[3]) for row in params]
print('MSE', np.min(values))



In [ ]:
# Iterations involve batch random sampling at local and globally bounded levels.

ITERATIONS = 25 # Each iteration may take several seconds.
EXPLORATION_PTS = 2000 # Run this many random explorations per step
LOCAL_SEARCH_PER_TOP_K = 100 # Run this many local explorations per candidate point
TOP_K = 20 # Maintain top K optimal points as local search areas and bounding box for new search region
reg = 0.1 # Regularization: stretch the boundaries of the bounding box by this fraction every iteration for exploration
local_search_range_multiplier = 0.001 # This fraction of the total boundary length is used for local search

for iters in range(ITERATIONS):

    top_k_indices = np.argsort(values)[:TOP_K]
    top_k_params = params[top_k_indices]
    top_k_values = [values[k] for k in top_k_indices]
    top_value = np.min(values)
    print('Iteration', iters)
    print('Best MSE so far', top_value)

    # Compute unbounded top_k bounding box to get range of expansion of bounding box.
    top_k_min_bounds = np.min(top_k_params, axis=0)
    top_k_max_bounds = np.max(top_k_params, axis=0)
    top_k_range = (top_k_max_bounds - top_k_min_bounds) / 2
    # Bound expanded bounding box by the initial bounding box so box will not go out of maximum search range.
    top_k_min_bounds = np.maximum(top_k_min_bounds - top_k_range * reg, min_bounds)
    top_k_max_bounds = np.minimum(top_k_max_bounds + top_k_range * reg, max_bounds)
    top_k_range = (top_k_max_bounds - top_k_min_bounds) / 2
    print('New death_rate_interval', top_k_min_bounds[0], top_k_max_bounds[0])
    print('New contacts_per_day_interval', top_k_min_bounds[1], top_k_max_bounds[1])
    print('New days_to_recover_interval', top_k_min_bounds[2], top_k_max_bounds[2])
    print('New case_fatality_multiplier_interval', top_k_min_bounds[3], top_k_max_bounds[3])

    # Do some local search
    for i in range(TOP_K):
        tmp_params = top_k_params[i]
        local_min_bounds = np.maximum(min_bounds, tmp_params - top_k_range)
        local_max_bounds = np.minimum(max_bounds, tmp_params + top_k_range)
        local_params = np.random.uniform(
            local_min_bounds, local_max_bounds, size=(LOCAL_SEARCH_PER_TOP_K, 4)
        )
        local_values = [mse_metric(row[0], row[1], row[2], row[3]) for row in local_params]
        best_local_value = np.min(local_values)
        if best_local_value < top_k_values[i]:
            if best_local_value < top_value:
                print('found new min {} in {}(th) best neighborhood'.format(best_local_value, len(top_k_values) - i))
            params[top_k_indices[i]] = local_params[np.argmin(local_values)]
            values[top_k_indices[i]] = best_local_value

    # Do some top-bounded exploration
    new_params = np.random.uniform(top_k_min_bounds, top_k_max_bounds, size=(EXPLORATION_PTS, 4))
    params = np.concatenate((params, new_params))
    values.extend([mse_metric(row[0], row[1], row[2], row[3]) for row in new_params])
    print('Total points explored', params.shape[0])
    print('MSE', np.min(values))


In [ ]:
#Validation plot
validation_area_df, _ = get_time_series_for_area(AREA_OF_INTEREST)

best_index = np.argmin(values)
best_params = params[best_index]
best_death_rate = best_params[0]
best_contacts = best_params[1]
best_days_to_recover = best_params[2]
best_case_fatality_multiplier = best_params[3]
best_mse = np.min(values)

infected = 0.04 * starting_cases * best_case_fatality_multiplier
t, s, i, r = compute_sir(
    SAMPLING_RATE,
    SIMULATION_DAYS,
    population * best_death_rate,
    infected,
    best_contacts,
    best_days_to_recover
)

print('best_death_rate:', best_death_rate)
print('best_contact:', best_contacts)
print('best_days_to_recover:', best_days_to_recover)
print('best_case_fatality_multiplier', best_case_fatality_multiplier)
print('best_mse', best_mse)
plot_sir_model(r, i, SIMULATION_DAYS, validation_area_df, SAMPLING_RATE, AREA_OF_INTEREST[1])

In [ ]:
mse_metric(0.0040, 0.35338, 24.0647, 0.1000000)